# Chicago Traffic Crashes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_crashes = pd.read_csv('crashes.csv')
df_vehicles = pd.read_csv('crashes_vehicles.csv').drop('CRASH_DATE',axis=1)

In [ ]:
#merge both tables after figuring out the common column and drop dupliacte columns before merging to reduce redundancy
df = pd.merge(df_crashes,df_vehicles,on='CRASH_RECORD_ID')

In [ ]:
#Get acquainted with the data
df.describe(include='object')

In [ ]:
#convert columns datatypes to improve efficiency
df['CRASH_DATE'] = pd.to_datetime(df['CRASH_DATE'])
df['WEATHER_CONDITION'] = df['WEATHER_CONDITION'].astype('category')
df['LIGHTING_CONDITION'] = df['LIGHTING_CONDITION'].astype('category')
df['PRIM_CONTRIBUTORY_CAUSE'] = df['PRIM_CONTRIBUTORY_CAUSE'].astype('category')
df['FIRST_CRASH_TYPE'] = df['FIRST_CRASH_TYPE'].astype('category')
df['HIT_AND_RUN_I'] = df['HIT_AND_RUN_I'].astype('category')
df['LIC_PLATE_STATE'] = df['LIC_PLATE_STATE'].astype('category')
df['VEHICLE_TYPE'] = df['VEHICLE_TYPE'].astype('category')
df['MAKE'] = df['MAKE'].astype('category')

In [ ]:
# Total number of reported crashes
print('Total number reported cases are {} as per the dataset'.format(df['CRASH_RECORD_ID'].nunique()))

In [ ]:
#Drop observations that did not occur in 2018, 2019 or 2020 (other years have incomplete data)
df = df[(df['CRASH_DATE']>='2018-01-01')&(df['CRASH_DATE']<='2020-12-31')]

In [ ]:
df.info()

In [ ]:
#Display a plot showing the number of crashes that occur for each hour of the day.
sns.set_theme(style='darkgrid')
df['HOUR'] = df['CRASH_DATE'].dt.hour
plt.figure(figsize=(10,5))
a = df.groupby('HOUR')['CRASH_RECORD_ID'].nunique().reset_index()
sns.barplot(data=a,y='CRASH_RECORD_ID',x='HOUR',)
plt.xlabel('Hour of the day')
plt.ylabel('No of accidents')
plt.title('Hourly Number of Reported Crashes in Chicago (2018-2020)')

In [ ]:
#Name the make of vehicle that was involved in the most daylight crashes in August 2018. Remember that a crash can involve multiple vehicles.
df_daylight = df[df['LIGHTING_CONDITION'] == 'DAYLIGHT']
df_daylight.groupby('MAKE')['CRASH_RECORD_ID'].count().sort_values().tail(1)
#ALTERNATE SOLUTION 
df_daylight['MAKE'].value_counts().nlargest(1)

In [ ]:
#Determine which weather condition was most prevalent for each type of crash.
df_crashes.groupby('FIRST_CRASH_TYPE')['WEATHER_CONDITION'].apply(lambda x: x.value_counts().head(1)).reset_index(name='count').rename(columns ={'level_1':'WEATHER'})       

In [ ]:
#Plot the primary contributing cause of reported crashes, from highest to lowest.
plt.figure(figsize=(20,20))
sns.set(font_scale=2)
sns.countplot(y='PRIM_CONTRIBUTORY_CAUSE',order = df['PRIM_CONTRIBUTORY_CAUSE'].value_counts().index,data=df)


In [ ]:
#Display the 10 state license plates involved in the most crashes. Remember that a crash can involve multiple vehicles.
df.groupby('LIC_PLATE_STATE')['VEHICLE_ID'].count().nlargest(10).reset_index(name='COUNT')

In [ ]:
#Display the proportion of crashes in each month of 2019 where alcohol was determined to be the primary contributing cause.
#total number of accidents month wise in 2019
df_2019 = df[(df['CRASH_DATE']>='2019-01-01')&(df['CRASH_DATE']<='2019-12-31')]
a = df_2019.groupby(df_2019['CRASH_DATE'].dt.month)['CRASH_RECORD_ID'].nunique().reset_index(name='COUNT')
# number of accidents caused by alcohol in 2019
df_alcohol = df_2019[df_2019['PRIM_CONTRIBUTORY_CAUSE'].str.contains('ALCOHOL|DRINKING')]
b = df_alcohol.groupby(df_alcohol['CRASH_DATE'].dt.month)['CRASH_RECORD_ID'].nunique().reset_index(name='COUNT')
# percentage calculation
percent = ((b['COUNT']/a['COUNT'])*100.)
c = percent.to_frame().rename(columns={'COUNT':'PERCENT'})
c.set_index(b.CRASH_DATE)


In [ ]:
#Determine whether snowmobiles or recreational off-highway vehicles were involved in more crashes.
a = len(df[df['VEHICLE_TYPE']=='SNOWMOBILE'])
b = len(df[df['VEHICLE_TYPE']=='RECREATIONAL OFF-HIGHWAY VEHICLE (ROV)'])
if a>b:
    print('Snowmobiles were involved in more accidents')
else:
    print('Recreational off-highway vehicles were involved in more accidents')